# Politics in Swarthmore

In [7]:
SCHOOL="swarthmore"
SUBJECT="opinions"
START_YEAR=2010
FINAL_YEAR=2022
DATA_DIR="data"      # should be 'data'
OUTPUT_DIR="output"  # should be 'output'

In [2]:
%pwd

'/Users/theodoremui/Library/CloudStorage/OneDrive-Personal/dev/diversity-colleges/notebooks'

In [3]:
%cd ..

/Users/theodoremui/Library/CloudStorage/OneDrive-Personal/dev/diversity-colleges


In [4]:
import sys
sys.path.append('src')
import ouraws
import ourgraphs
import textutil

In [8]:
# NOTE: before loading, we need to be in the "diversity-colleges" folder

S3OBJECT_KEY = f"{DATA_DIR}/{SCHOOL}-{SUBJECT}-SNAPSHOT.parquet"
df = ouraws.getFromS3(S3OBJECT_KEY)

In [9]:
df.shape

(2064, 6)

In [10]:
df.head(2)

,title,url,body,year,month,day
49,The Anti-War Left Should Stop Westplaining,https://swarthmorephoenix.com/2022/03/17/the-a...,The full-scale Russian invasion of Ukraine has...,2022,3,17
50,The Anti-War Left Should Stop Westplaining,https://swarthmorephoenix.com/2022/03/17/the-a...,The full-scale Russian invasion of Ukraine has...,2022,3,17


In [15]:
import requests
import os

BIPARTISAN_API_KEY = os.environ.get("BIPARTISAN_API_KEY")

BIPARTISAN_URL = "https://api.thebipartisanpress.com/api/endpoints/beta/robert"

In [16]:
BIPARTISAN_API_KEY

'gAAAAABeVpQJKRM5BqPX91XW2AKfz8pJosk182maAweJcm5ORAkkBFj__d2feG4H5KIeOKFyhUVSY_uGImiaSBCwy2L6nWxx4g=='

In [17]:
df2022 = df[df['year'] == 2022]

articles_list = df2022['body'].to_list()
for article in articles_list:
    payload = {"API": BIPARTISAN_API_KEY, "Text": article.encode("utf-8")}
    response = requests.post(BIPARTISAN_URL, data=payload)
    print(response.text)
    break

-20.633898


In [18]:
df['year'].unique()

array([2022, 2021, 2020, 2019, 2018, 2017, 2016, 2015, 2014, 2013, 2012,
       2011, 2010, 2009, 2008, 2007, 2006, 2005])

In [19]:
# SYNCHRONOUS METHOD : TAKES A LONG TIME

results = []

for year in range(2010, 2023):
    df2 = df[df['year'] == year]
    print(f"Year: {year} ... {df2.shape[0]}")

    articles_list = df2['body'].to_list()
    value_sum = 0.0
    article_count = 0
    for article in articles_list:
        payload = {"API": BIPARTISAN_API_KEY, "Text": article.encode("utf-8")}
        response = requests.post(BIPARTISAN_URL, data=payload)
        try:
            value_sum += float(response.text)
            article_count += 1
            # print(f"{article_count}", end=".")
        except:
            pass  # this is a non-number
        
    results.append({
        'year' : year,
        'article_count' : article_count,
        'polarity_sum' : value_sum,
        'polarity_avg' : float(value_sum) / article_count
    })

    print(f"{year}\t{article_count}\t{value_sum}\t{value_sum/article_count}")
# for each year, get all articles' bodies & send to bipartisan press API 
#    and sum up the values & output the average for each year

Year: 2010 ... 69
2010	69	96.992445	1.4056876086956522
Year: 2011 ... 134
2011	131	-59.835691000000004	-0.45676100000000003
Year: 2012 ... 264
2012	261	-202.996644	-0.7777649195402299
Year: 2013 ... 203
2013	200	-428.4380890000002	-2.142190445000001
Year: 2014 ... 186
2014	176	-610.5524769999997	-3.4690481647727256
Year: 2015 ... 208


In [ ]:
import pandas as pd

RESULTS_FILE = f"{DATA_DIR}/{SCHOOL}-POLARITY.parquet"

results_df = pd.DataFrame.from_records(results)
results_df.to_parquet(RESULTS_FILE)